In [1]:
# import services.theme_extractor_api.main
from services.theme_extractor.cluster_job import ClusterJob
from services.theme_extractor.wv_model_job import WVModelJob

from services.theme_extractor.article_preprocess_job import ArticlePreprocessJob




In [2]:

apj = ArticlePreprocessJob()

raw_articles = apj.get_articles_for_latest_load()[:10000]


In [3]:
processed_articles = apj.preprocess_raw_articles(raw_articles)

In [6]:
processed_articles[0].words

['scotland_yard',
 'investigate',
 'claim',
 'worker',
 'outsource',
 'firm',
 'caput',
 'pay',
 'convict',
 'deliberately',
 'fit',
 'electronic',
 'ankle',
 'tag',
 'loosely',
 'allow',
 'slip',
 'device',
 'when',
 'want',
 'go',
 'staff',
 'company',
 'run',
 'government',
 'electronic_monitoring',
 'service',
 'allegedly',
 'pay',
 'time',
 'help',
 'at_least',
 'offender',
 'beat',
 'court',
 'impose',
 'curfew',
 'accord_report',
 'sun',
 'metropolitan_police',
 'say',
 'investigation',
 'centre',
 'london_borough',
 'newham',
 'say',
 'people',
 'include',
 'current',
 'former',
 'ems',
 'worker',
 'arrest',
 'connection',
 'offence',
 'involve',
 'monitoring',
 'offender',
 'accord',
 'sun',
 'scheme',
 'reveal',
 'offender',
 'arrest_suspicion',
 'attempt_murder',
 'suppose',
 'home',
 'curfew',
 'electronic_tag',
 'use',
 'monitor',
 'condition',
 'court',
 'prison',
 'order',
 'usually',
 'securely',
 'attach',
 'ankle',
 'defender',
 'can',
 'remove',
 'then',
 'send',
 'l

In [7]:
from gensim.models.phrases import Phrases
from typing import List
import numpy as np

tokenized_texts = apj.preprocessor.preprocessed_docs


def get_ngrams(tokenized_texts: List[List[str]], phrases: Phrases):
    return np.unique([token for token in sum([phrases[phrases[tokens]] for tokens in tokenized_texts ], []) if '_' in token])

def comp_phrasers():
    
    phrases1 = Phrases(tokenized_texts)

    phrases2 = Phrases(tokenized_texts, scoring='npmi', threshold=0.2, min_count=50, progress_per=1000)
        
    tokens1 = get_ngrams(tokenized_texts, phrases1)
    tokens2 = get_ngrams(tokenized_texts, phrases2)

    
    in1butnot2 = np.setdiff1d(tokens1, tokens2, assume_unique=True)
    in2butnot1 = np.setdiff1d(tokens2, tokens1, assume_unique=True)

    return tokens1, tokens2, in1butnot2, in2butnot1

tokens1, tokens2, unique_to_1, unique_to_2 = comp_phrasers()

In [11]:
print(len(unique_to_2))
for token in token:
    print(token)

el_agent
travel_allowance
travel_courtesy
travel_disruption
travel_expense
travel_extensively
travel_freely
travel_mph
travel_restriction
traveller_muslim
travis_county
travis_kalanick
treason_charge
treasurer_joe
treasurer_scott
treasury_official
treasury_secretary
treasury_select
treat_differently
treat_equally
treat_fairly
treat_injury
treat_patient
treaty_european
treaty_obligation
treaty_rome
tree_fell
tree_plant
tremendous_amount
tremendous_respect
trevor_bayliss
trevor_noah
trevor_timm
tribal_chairman
tribal_council
tribal_leadership
tribe_fear
tribe_member
tribunal_fee
tribunal_hear
tribunal_judge
tribunal_ruling
trident_missile
trident_nuclear
trident_test
trigger_backlash
trigger_brexit
trigger_resignation
trigger_widespread
trillion_dollar
trinity_college
trinity_mirror
trip_gold
triple_credit
triple_lock
tristram_hunt
triumph_hurdle
trojan_horse
trolley_corridor
trolley_wait
tronc_system
trophy_hunting
tropical_low
tropical_medicine
trot_conspiracy
troy_deeney
troy_grant
tr

In [40]:
import en_core_web_sm
nlp = en_core_web_sm.load(disable=['ner', 'parser'])
from bs4 import BeautifulSoup

def extract_text_from_html(res: str) -> str:
    soup = BeautifulSoup(res, features="lxml")
    
    for f in soup.find_all('figure'):
        f.decompose()
    
    text = soup.get_text().lower();
    
    return text

text = extract_text_from_html(raw_articles[0].body)